In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_curve, auc, fbeta_score
from sklearn.model_selection import train_test_split
from opt import RAdam
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '8'

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

import efficientnet.keras as efn
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Add, BatchNormalization, Input, Dense, MaxPooling2D, Conv2D, Flatten, Concatenate
from keras.layers.core import Activation, Layer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

import keras.backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))


/home/cenpes/anaconda3/envs/DL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cenpes/anaconda3/envs/DL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cenpes/anaconda3/envs/DL/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cenpes/anaconda3/envs/DL/lib/python3.7/site-packages

In [2]:
print("Importing Dataset")

data_dir = '/home/dados4t/DataChallenge2/'

images = np.load(os.path.join(data_dir,'images_efn_vis.npy'))
is_lens = np.load(os.path.join(data_dir,'Y.npy'))
#pad = np.zeros((images.shape[0],images.shape[1],images.shape[2],1), dtype="float32")

X_train, X_test, Y_train, Y_test = train_test_split(images, is_lens, test_size = 0.10, random_state = 7)
del images
print(X_train.shape)
print(X_test.shape)


Importing Dataset
(89991, 200, 200, 3)
(10000, 200, 200, 3)


In [46]:
print("Building Model")

inp = Input((200,200,3))
efn_arc = efn.EfficientNetB2(input_tensor = inp, weights='imagenet')

y_hat = Dense(2,activation ="sigmoid")(efn_arc.layers[-2].output)

model = Model(efn_arc.input, y_hat)



Building Model


In [47]:
batch_size = 35
def fbeta(y_true, y_pred):
    TP = (K.sum((y_pred * y_true), axis=-1)) / batch_size
    FP = (K.sum(((1 - y_pred) * y_true), axis=-1)) / batch_size
    FN = (K.sum((y_pred * (1 - y_true)), axis=-1)) / batch_size
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    fbeta = (1 + 0.001) * precision * recall / ( 0.001 * precision + recall)
    fbeta = 1 - K.mean(fbeta)
    return fbeta

In [48]:
#model.compile(loss = 'categorical_crossentropy', optimizer=RAdam(),metrics = ['accuracy'])

In [49]:
model.compile(loss = fbeta, optimizer=RAdam(clipnorm=0.001),metrics = ['accuracy'])

In [ ]:
print("Training Model")

model_name = "efn2_vis_TEST.hdf5"
batch_size = 35
check = ModelCheckpoint(model_name, monitor="val_loss", verbose=1, save_best_only=True)

gen = ImageDataGenerator(
		rotation_range=180,
		zoom_range=0.20,
		vertical_flip = True,
    horizontal_flip=True,
		fill_mode="nearest")
"""
def gen_flow_for_three_inputs(X1, X2, X3, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=1)
    genX2 = gen.flow(X2, batch_size=batch_size,seed=1)
    genX3 = gen.flow(X3, batch_size=batch_size,seed=1)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i, X3i], X1i[1]

gen_flow = gen_flow_for_three_inputs(X_train_h, X_train_j, X_train_y, Y_train)
"""

history = model.fit_generator(gen.flow(X_train, Y_train, batch_size = batch_size), epochs = 20,  
            verbose = 1, validation_data= (X_test, Y_test), callbacks=[check], 
            steps_per_epoch = X_train.shape[0] // batch_size)


Training Model
Epoch 1/20
 433/2571 [====>.........................] - ETA: 53:43 - loss: 0.0414 - acc: 0.5030